### 1. Import libraries and load data

In [1]:
import pandas as pd

# Load data from csv files
content = pd.read_csv("Content.csv")
reaction_types = pd.read_csv('ReactionTypes.csv')
reactions = pd.read_csv('Reactions.csv')

### 2. Inspect, Delete obsolete columns and Create dataframe

In [2]:
# Droping unneeded columns, renaming and checking data types
content.drop(['Unnamed: 0','User ID','URL'],axis=1, inplace=True)
print(content.info())

reactions.drop(['Unnamed: 0','User ID'], axis=1, inplace=True)
print(reactions.info())

reaction_types.drop('Unnamed: 0', axis=1, inplace=True)
print(reaction_types.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Content ID  1000 non-null   object
 1   Type        1000 non-null   object
 2   Category    1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25553 entries, 0 to 25552
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Content ID  25553 non-null  object
 1   Type        24573 non-null  object
 2   Datetime    25553 non-null  object
dtypes: object(3)
memory usage: 599.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Type       16 non-null     object
 1   Sentiment  16 non-null     object
 2   Score      16 non-null     int6

In [3]:
# Create a Dataframe by merging data
reactions = reactions.rename(columns={"Type":"Reaction_type"})


df = reactions.merge(content, how="left", on="Content ID")
df = df.rename(columns={'Type_x':'Reaction_type','Type':'Content_type'})
print(df.head(8))

reaction_types = reaction_types.rename(columns={"Type":"Reaction_type"})
df = df.merge(reaction_types, how="left", on='Reaction_type')
print(df.head(8))


print(df.isnull().sum()) # checking for null values

                             Content ID Reaction_type             Datetime  \
0  97522e57-d9ab-4bd6-97bf-c24d952602d2           NaN  2021-04-22 15:17:15   
1  97522e57-d9ab-4bd6-97bf-c24d952602d2       disgust  2020-11-07 09:43:50   
2  97522e57-d9ab-4bd6-97bf-c24d952602d2       dislike  2021-06-17 12:22:51   
3  97522e57-d9ab-4bd6-97bf-c24d952602d2        scared  2021-04-18 05:13:58   
4  97522e57-d9ab-4bd6-97bf-c24d952602d2       disgust  2021-01-06 19:13:01   
5  97522e57-d9ab-4bd6-97bf-c24d952602d2    interested  2020-08-23 12:25:58   
6  97522e57-d9ab-4bd6-97bf-c24d952602d2       peeking  2020-12-07 06:27:54   
7  97522e57-d9ab-4bd6-97bf-c24d952602d2       cherish  2021-04-11 17:35:49   

  Content_type  Category  
0        photo  Studying  
1        photo  Studying  
2        photo  Studying  
3        photo  Studying  
4        photo  Studying  
5        photo  Studying  
6        photo  Studying  
7        photo  Studying  
                             Content ID Reaction_type 

### 3. Data Cleaning

In [4]:
# Delete rows with null values in content_type
df = df.dropna(subset=["Content_type"])
df.info()

# Identify unique categories
print(df['Category'].unique())
print(df['Category'].nunique())


# format quotations to single quotations
df['Category'] = df.Category.replace('"','',regex=True)
df['Category'] = df.Category.str.lower()
print(df['Category'].nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25553 entries, 0 to 25552
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Content ID     25553 non-null  object 
 1   Reaction_type  24573 non-null  object 
 2   Datetime       25553 non-null  object 
 3   Content_type   25553 non-null  object 
 4   Category       25553 non-null  object 
 5   Sentiment      24573 non-null  object 
 6   Score          24573 non-null  float64
dtypes: float64(1), object(6)
memory usage: 1.6+ MB
['Studying' 'healthy eating' 'technology' 'food' 'cooking' 'dogs' 'soccer'
 'public speaking' 'science' 'tennis' 'travel' 'fitness' 'education'
 'studying' 'veganism' 'Animals' 'animals' 'culture' '"culture"' 'Fitness'
 '"studying"' 'Veganism' '"animals"' 'Travel' '"soccer"' 'Education'
 '"dogs"' 'Technology' 'Soccer' '"tennis"' 'Culture' '"food"' 'Food'
 '"technology"' 'Healthy Eating' '"cooking"' 'Science' '"public speaking"'
 '"veganism"

### 4. Top 5 Aggregate Categories

In [5]:
# Using the SQL logic below with pandas
# SELECT  *, sum(score) as 'Top 5 Categories' FROM df
# WHERE content_type is not null
# group by 1
# order by 2 desc
# limit 5

df.to_csv('clean_accentdata.csv') # Cleaned data csv

''' Note: NaN values in sentiment,reactions & scores are Structurally Missing Data since without a reaction, there's no sentiment or score that can be associated with the content '''

sum_cat_df = df.groupby('Category').Score.sum() # total scores in each category
sum_cat_df = sum_cat_df.sort_values(ascending=False)

print(sum_cat_df.head())

Category
animals           74965.0
science           71168.0
healthy eating    69339.0
technology        68738.0
food              66676.0
Name: Score, dtype: float64


In [6]:
reaction_types = pd.read_csv('ReactionTypes.csv')
reaction_types

,Unnamed: 0,Type,Sentiment,Score
0,0,heart,positive,60
1,1,want,positive,70
2,2,disgust,negative,0
3,3,hate,negative,5
4,4,interested,positive,30
5,5,indifferent,neutral,20
6,6,love,positive,65
7,7,super love,positive,75
8,8,cherish,positive,70
9,9,adore,positive,72


In [7]:
from lets_plot import * 
ggplot() + \
geom_point(aes(x="Type", y="Score", color="Sentiment"), data=reaction_types, sampling="none" if reaction_types.size < 2500 else sampling_systematic(n=2500)) + \
ggtitle("Reaction types") 